In [1]:
import pandas as pd
import cobra
from cobra.io import load_json_model
from cobra.flux_analysis import single_gene_deletion, single_reaction_deletion, double_gene_deletion
import numpy as np

import re
from glob import glob
from Bio import Entrez, SeqIO

# I. Gene comparison EcN aerobic vs. anaerobic

In [2]:
# Load EcN model
EcN_ID = 'CP022686.1'
EcN_model = cobra.io.load_json_model('../data/models/%s_cur_4.8.json'%EcN_ID)

In [3]:
#Establish a definition that initializes models to an in silico representation of gut microbiome media
def M1(model):
    
    medium = model.medium
    
    # Set exchange to zero
    for rxn in medium:
        medium[rxn] = 0.0 # Remove all current media components

    # Read medias - FROM AGORA
    media_db = pd.read_table("../tables/media_db.txt", comment="#")
    media_db["compound"] = [cpd.strip() for cpd in media_db["compound"]]
    media_db = media_db[["medium", "compound"]].groupby("medium").agg(lambda x: list(x)).to_dict()['compound']


    # Add compounds to media composition
    for compound in media_db['M1']: # M1 > Gut microbium media, GMM
        try:
            model.reactions.get_by_id('EX_%s_e'%compound)
            medium['EX_%s_e'%compound]= 1000
        except:
#             print(compound, 'is missing in the model')
            pass
    
    medium['EX_o2_e']= 1000 # Make aerobic, so we can analyse the differences between aerobic and anaerobic
        
    # Update medium
    model.medium = medium
    
    return model

In [4]:
# Get growth rate for each single gene knockout
with EcN_model:
    M1(EcN_model)
    wt_growth_ae = EcN_model.optimize().objective_value
    ae_results = single_gene_deletion(EcN_model) 
    
with EcN_model:
    M1(EcN_model)
    
    # Make anaerobic
    medium = EcN_model.medium
    medium['EX_o2_e']= 0.0
    EcN_model.medium = medium
    
    wt_growth_an = EcN_model.optimize().objective_value
    an_results = single_gene_deletion(EcN_model)

ae_results.head()

,ids,growth,status
0,{CIW80_25350},3.365587e-10,optimal
1,{CIW80_24880},3.555409e+01,optimal
2,{CIW80_15725},3.555409e+01,optimal
3,{CIW80_23550},3.555409e+01,optimal
4,{CIW80_06285},3.555409e+01,optimal


In [5]:
# Get binary representation of gene essentiality
ae_results['binary_ae']= ae_results['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_ae) else 1)
an_results['binary_an']= an_results['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_an) else 1)

In [6]:
# Check whether there are genes for which the status is not "optimal"
ae_results[ae_results.status != 'optimal']

,ids,growth,status,binary_ae


In [7]:
# Check whether there are genes for which the status is not "optimal"
infeasible_an = an_results[an_results.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in infeasible_an:
    an_results.loc[gene, 'binary_an'] = 0
    
an_results[an_results.status != 'optimal']

,ids,growth,status,binary_an


In [8]:
# Unpack the index values from the frozenset to enable comparison to the ortho_matrices
for del_result in [ae_results, an_results]:
    for value in del_result.index:
        unpacked, = del_result.loc[value, 'ids']
        del_result.loc[value, 'gene'] = unpacked

    # Set the unpacked gene name as index
    del_result.set_index('gene', inplace=True)

In [9]:
# Remove the other two columsn
ae_results.drop(columns=['growth', 'status'], inplace=True)
an_results.drop(columns=['growth', 'status'], inplace=True)

# Merge the two dataframes and 
del_results = pd.merge(ae_results, an_results, left_index=True, right_index=True)

# Get an overview of the total number of essential genes
print('total =', len(del_results))
print('Aerobic essential =', len(del_results) - del_results['binary_ae'].sum())
print('Anaerobic essential =', len(del_results) - del_results['binary_an'].sum())

# Identify the genes that are only essential in one of the two conditions
del_results.drop(columns=['ids_x', 'ids_y'], inplace=True) # Drop the ids columns
del_results['diff'] = del_results['binary_ae'] - del_results['binary_an']
del_results[del_results['diff'] != 0]

total = 1533
Aerobic essential = 155
Anaerobic essential = 156


,binary_ae,binary_an,diff
gene,,,
CIW80_14315,1,0,1


In [10]:
del_results_ae = del_results[del_results['binary_ae'] == 0]
del_results_an = del_results[del_results['binary_an'] == 0]

# Save the resulting dataframe as .csv
del_results.to_csv('../tables/gene_ess_del_results.csv')

In [11]:
print(del_results.sum())

binary_ae    1378
binary_an    1377
diff            1
dtype: int64


In [12]:
# Find out which genes were essential only in anaerobic conditions
for gene in del_results[del_results['diff'] != 0].index:
    name = EcN_model.genes.get_by_id(gene).name
    print(name)

hemN


### Investigate hemN

In [13]:
EcN_model.genes.CIW80_14315

Gene identifier,CIW80_14315
Name,hemN
Memory address,0x021dbde65308
Functional,True
In 1 reaction(s),CPPPGO2


In [14]:
import escher
from escher import Builder

In [15]:
builder = Builder(
    map_name='iJO1366.Central metabolism',
    model_name='iJO1366')

In [16]:
builder

Builder()

In [17]:
with EcN_model:
    M1(EcN_model)
    EcN_model.reactions.EX_o2_e.lower_bound=0
    solution = EcN_model.optimize()

In [18]:
builder.reaction_data = solution.fluxes

In [19]:
EcN_model.genes.CIW80_14315

Gene identifier,CIW80_14315
Name,hemN
Memory address,0x021dbde65308
Functional,True
In 1 reaction(s),CPPPGO2


In [20]:
EcN_model.genes.CIW80_06055

Gene identifier,CIW80_06055
Name,hemF
Memory address,0x021dbddf1188
Functional,True
In 1 reaction(s),CPPPGO


In [21]:
EcN_model.reactions.CPPPGO2

Reaction identifier,CPPPGO2
Name,Oxygen Independent coproporphyrinogen-III oxidase
Memory address,0x021dc0d4ee08
Stoichiometry,2.0 amet_c + cpppg3_c --> 2.0 co2_c + 2.0 dad_5_c + 2.0 met__L_c + pppg9_c 2.0 S-Adenosyl-L-methionine + Coproporphyrinogen III --> 2.0 CO2 CO2 + 2.0 5'-Deoxyadenosine + 2.0 L-Methionine + Protoporphyrinogen IX
GPR,CIW80_14315
Lower bound,0.0
Upper bound,1000.0


In [22]:
EcN_model.reactions.CPPPGO

Reaction identifier,CPPPGO
Name,Coproporphyrinogen oxidase (O2 required)
Memory address,0x021dbdf3cbc8
Stoichiometry,cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c Coproporphyrinogen III + 2.0 H+ + O2 O2 --> 2.0 CO2 CO2 + 2.0 H2O H2O + Protoporphyrinogen IX
GPR,CIW80_06055
Lower bound,0.0
Upper bound,1000.0


The oxygen-dependency of hemF makes it unable to rescue grown anaerobically for an hemN knockout strain.

# II. Essential reactions EcN & subsystem

In [23]:
# Get growth rate for each single gene knockout
with EcN_model:
    M1(EcN_model)
    EcN_model.reactions.EX_o2_e.lower_bound=0 # Check essential reactions for anaerobic conditions
    wt_growth_an = EcN_model.optimize().objective_value
    an_rxn = single_reaction_deletion(EcN_model)

# Get binary representation of gene essentiality
an_rxn['binary']= an_rxn['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_an) else 1)

# Check whether there are genes for which the status is not "optimal"
inf_rxn_an = an_rxn[an_rxn.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in inf_rxn_an:
    an_rxn.loc[gene, 'binary'] = 0
an_rxn[an_rxn.status != 'optimal']

,ids,growth,status,binary


In [24]:
# Unpack the index values from the frozenset to enable comparison to the ortho_matrices
for value in an_rxn.index:
    unpacked, = an_rxn.loc[value, 'ids']
    an_rxn.loc[value, 'reaction'] = unpacked
    
# Set the unpacked gene name as index
an_rxn = an_rxn.set_index('reaction')
an_rxn.rename(columns = {'binary':'an_rxn_growth'}, inplace=True)

# Get the subsystem for all reactions
for rxn in an_rxn.index.tolist():
    an_rxn.loc[rxn, 'Subsystem'] = EcN_model.reactions.get_by_id(rxn).subsystem
    
print('total =', len(an_rxn))
print('Essential =', len(an_rxn) - an_rxn.an_rxn_growth.sum())
print('Non-essential =', an_rxn.an_rxn_growth.sum())

an_rxn

total = 3143
Essential = 230
Non-essential = 2913


,ids,growth,status,an_rxn_growth,Subsystem
reaction,,,,,
ULA4Ntppi,{ULA4Ntppi},35.540644,optimal,1,Lipopolysaccharide Biosynthesis / Recycling
EX_feroxEfe_e,{EX_feroxEfe_e},35.540644,optimal,1,Extracellular exchange
ATPS4rpp,{ATPS4rpp},24.856463,optimal,1,Oxidative Phosphorylation
EX_cys__L_e,{EX_cys__L_e},35.252491,optimal,1,Extracellular exchange
PPRDNDH,{PPRDNDH},35.540644,optimal,1,Lysine Metabolism
...,...,...,...,...,...
ORNabcpp,{ORNabcpp},35.540644,optimal,1,"Transport, Inner Membrane"
FESR,{FESR},35.540644,optimal,1,Cofactor and Prosthetic Group Biosynthesis
EX_malt_e,{EX_malt_e},35.540644,optimal,1,Extracellular exchange


In [25]:
gene_ess_an_sub = an_rxn[an_rxn['an_rxn_growth'] == 0 ]['Subsystem'].value_counts()
gene_ess_an_sub

Cofactor and Prosthetic Group Biosynthesis            53
Purine and Pyrimidine Biosynthesis                    17
Cell Envelope Biosynthesis                            17
Tyrosine, Tryptophan, and Phenylalanine Metabolism    16
Glycerophospholipid Metabolism                        15
Valine, Leucine, and Isoleucine Metabolism            14
Extracellular exchange                                14
Lipopolysaccharide Biosynthesis / Recycling           12
Threonine and Lysine Metabolism                       12
Arginine and Proline Metabolism                       12
Methionine Metabolism                                  9
Transport, Outer Membrane                              9
Nucleotide Salvage Pathway                             5
Transport, Inner Membrane                              4
Citric Acid Cycle                                      4
Inorganic Ion Transport and Metabolism                 3
Membrane Lipid Metabolism                              2
Alanine and Aspartate Metabolis

In [26]:
gene_ess_an_sub.to_csv('../tables/gene_ess_an_sub.csv')

# VI. Double gene essentiality

In [ ]:
# Create a list of all non-essential genes
ae_non = del_results[del_results['binary_ae'] == 1].index.tolist()

# Run the double gene deletion with only the non-essential genes
with EcN_model:
    M1(EcN_model)
    double_ae = double_gene_deletion(EcN_model, ae_non, return_frame=True).round(4)
    
double_ae.head()

In [ ]:
# Get binary representation of gene essentiality
double_ae['binary']= double_ae['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_ae) else 1)

# Check whether there are genes for which the status is not "optimal"
inf_double = double_ae[double_ae.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in inf_double:
    double_ae.loc[gene, 'binary'] = 0
    
double_ae[double_ae.status != 'optimal']

In [ ]:
double_ess = double_ae[double_ae.status == 'optimal']
double_ess = double_ess[double_ess['binary'] == 0]
print(len(double_ess))
double_ess.head()

In [ ]:
# Save the resulting dataframe as .csv
double_ess.to_csv('../tables/gene_ess_double_ess.csv')

### Anaerobic

In [ ]:
# Create a list of all non-essential genes
an_non = del_results[del_results['binary_an'] == 1].index.tolist()

# Run the double gene deletion with only the non-essential genes
with EcN_model:
    M1(EcN_model)
    
    # Make anaerobic
    medium = EcN_model.medium
    medium['EX_o2_e']= 0.0
    EcN_model.medium = medium
    
    double_an = double_gene_deletion(EcN_model, an_non, return_frame=True).round(4)
    
double_an.head()

In [ ]:
# Get binary representation of gene essentiality
double_an['binary']= double_an['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_an) else 1)

# Check whether there are genes for which the status is not "optimal"
inf_double_an = double_an[double_an.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in inf_double_an:
    double_an.loc[gene, 'binary'] = 0
    
double_an[double_an.status != 'optimal']

In [ ]:
double_ess_an = double_an[double_an.status == 'optimal']
double_ess_an = double_ess_an[double_ess_an['binary'] == 0]
print(len(double_ess_an))
double_ess_an.head()

In [ ]:
# Save the resulting dataframe as .csv
double_ess_an.to_csv('../tables/gene_ess_double_ess_an.csv')

In [ ]:
# Get the names of the genes in the right format to compare
EcN_double = double_ess.copy()

for EcN_loc in EcN_double.index.tolist():
    EcN_id = EcN_double.loc[EcN_loc, 'ids']
    EcN_genes = []
    for gene in EcN_id:
        EcN_genes.append(str(gene))
    EcN_double.loc[EcN_loc, 'EcN'] = str(EcN_genes)

EcN_double_an = double_ess_an.copy()
for EcN_loc in EcN_double_an.index.tolist():
    EcN_id = EcN_double_an.loc[EcN_loc, 'ids']
    EcN_genes = []
    for gene in EcN_id:
        EcN_genes.append(str(gene))
    EcN_double_an.loc[EcN_loc, 'EcN'] = str(EcN_genes)

In [ ]:
# Create two sets
set_ae = set(EcN_double['EcN'])
set_an = set(EcN_double_an['EcN'])

diff_ae = set_ae - set_an
diff_an = set_an - set_ae
similar = set.intersection(set_ae, set_an)

print(len(diff_ae), len(diff_an), len(similar))

In [ ]:
def gene_names(gene_set):
    df = pd.DataFrame(columns=['EcN_id', 'EcN_id_1', 'EcN_id_2', 'EcN_name_1', 'EcN_name_2', 'EcN_names'])

    df['EcN_id'] = list(gene_set)
    for loc in df.index:
        
        #Split the name in two gene_ids
        name = df.loc[loc, 'EcN_id']
        name = name.strip("['").strip("]'").split("', '")
        df.loc[loc, 'EcN_id_1'] = name[0]
        df.loc[loc, 'EcN_id_2'] = name[1]
        
        # Identify the gene names
        df.loc[loc, 'EcN_name_1'] = EcN_model.genes.get_by_id(name[0]).name
        df.loc[loc, 'EcN_name_2'] = EcN_model.genes.get_by_id(name[1]).name
        name = sorted(name)
        df.loc[loc, 'EcN_names'] = str(EcN_model.genes.get_by_id(name[0]).name + ', ' + EcN_model.genes.get_by_id(name[1]).name)
        
    return df

In [ ]:
# Get the names of the aerobic double essential genes
gene_overview_ae = gene_names(diff_ae)
gene_overview_ae

In [ ]:
# Get the names of the anaerobic double essential genes
gene_overview_an = gene_names(diff_an)
gene_overview_an

The gene combination "iscU/sufD", "trxB/gor" and "xdhC/guaB" are present in both, but in different order. Only "hemF/hemN" is only found in aerobic conditions and "rpe/pgl", "rpe/zwf", "carA/ybcF" and "carB/ybcF" are anaerobic specific.

##### Load the tables not to rerun the double_gene_deletion

In [ ]:
# Load if needed
double_ess = pd.read_csv('../tables/gene_ess_double_ess.csv')
double_ess.set_index('Unnamed: 0', inplace=True)
double_ess_an = pd.read_csv('../tables/gene_ess_double_ess_an.csv')
double_ess_an.set_index('Unnamed: 0', inplace=True)

In [ ]:
# get usable gene names
for EcN_loc in double_ess.index.tolist():
    gene_list = double_ess.loc[EcN_loc, 'ids'].split("', '")
    gene_1 = gene_list[0][2:]
    gene_2 = gene_list[1][:-2]
    double_ess.loc[EcN_loc, 'genes'] = gene_1 + ', ' + gene_2
    
for EcN_loc in double_ess_an.index.tolist():
    gene_list = double_ess_an.loc[EcN_loc, 'ids'].split("', '")
    gene_1 = gene_list[0][2:]
    gene_2 = gene_list[1][:-2]
    double_ess_an.loc[EcN_loc, 'genes_an'] = gene_1 + ', ' + gene_2
    
# Create two sets
set_ae = set(double_ess['genes'])
set_an = set(double_ess_an['genes_an'])

diff_ae = set_ae - set_an
diff_an = set_an - set_ae
similar = set.intersection(set_ae, set_an)

print('Total number of synthetic lethals aerobically:', len(set_ae), 'and anaerobically:', len(set_an))
print('The difference is:', len(diff_ae), len(diff_an), len(similar))
diff_ae

### Check differences

In [ ]:
# Get the reactions
EcN_model.genes.CIW80_06055 #hemF
EcN_model.genes.CIW80_14315 #hemN

In [ ]:
# Check the double lethal only for aerobic
with EcN_model:
    M1(EcN_model)
    EcN_model.reactions.EX_o2_e.lower_bound=0
    EcN_model.remove_reactions(['CPPPGO2', 'CPPPGO'])
    print(EcN_model.slim_optimize())

This deletion is lethal in both cases, but was not included for anaerobic, as the deletion of only CIW80_14315 (hemN/CPPPGO2) is already lethal in anaerobic conditions

# VII. Gene comparison EcN & MG1655 model

In [ ]:
# Load iML1515 model
k12_model = cobra.io.load_json_model('../data/models/iML1515.json')

# Get growth rate for each single gene knockout
with k12_model:
    M1(k12_model)
    k12_model.reactions.EX_o2_e.lower_bound=0
    del_result_k12 = single_gene_deletion(k12_model)

In [ ]:
# Get a binary representation of the gene essentiality
k12_growth = k12_model.optimize().objective_value

del_result_k12['binary']= del_result_k12['growth'].map(lambda x: 0 if x < (0.05 * k12_growth) else 1)

In [ ]:
# Check whether there are genes for which the status is not "optimal"
del_result_k12[del_result_k12.status != 'optimal']

In [ ]:
# Unpack the index values from the frozenset to enable comparison to the ortho_matrices
for value in del_result_k12.index:
    unpacked, = del_result_k12.loc[value, 'ids']
    del_result_k12.loc[value, 'gene'] = unpacked
                                   

# Set the unpacked gene name as index
del_result_k12 = del_result_k12.set_index('gene')
del_result_k12.rename(columns = {'binary':'K12_growth'}, inplace=True)
del_result_k12.drop(columns=['ids', 'growth', 'status'], inplace=True)
del_result_k12.head()

In [ ]:
# Get the ortho matrix of EcN vs. the reference strains
orthoIDs_matrix = pd.read_csv('../tables/orthoIDs_matrix.csv')
orthoIDs_matrix.set_index('Unnamed: 0', inplace=True)
geneIDs_sub = orthoIDs_matrix.loc[:, 'U00096'] # Get the gene ID orthologs for MG1655

# Merge the ortho_matrix with the EcN gene essentiality matrix
geneIDs_ess = pd.merge(geneIDs_sub, del_results, left_index = True, right_index = True)
geneIDs_ess.head()

In [ ]:
# Combine with previously generated EcN essentiality data merged with K12 gene IDs
ess_comp = pd.merge(geneIDs_ess, del_result_k12, left_on ='U00096', right_index =True)

# Find the differences
ess_comp['difference'] = ess_comp['binary_an'] - ess_comp['K12_growth']
ess_comp[ess_comp['difference'] != 0] 

In [ ]:
# Save the resulting dataframe as .csv
ess_comp_k12 = ess_comp#[ess_comp['K12_growth'] == 0]
# ess_comp_k12.to_csv('../tables/gene_ess_comp_k12.csv')

# Get the number of shared genes
len(ess_comp[ess_comp['U00096'].str.contains('b')])

In [ ]:
for gene in ess_comp[ess_comp['difference'] != 0]['U00096']:
    name = k12_model.genes.get_by_id(gene).name
    print(name)

In [ ]:
for gene in ess_comp[ess_comp['difference'] != 0].index:
    name = EcN_model.genes.get_by_id(gene).name
    print(name)

In [ ]:
k12_model.genes.b1912

In [ ]:
EcN_model.genes.CIW80_02775

with EcN_model:
    M1(EcN_model)
#     EcN_model.reactions.EX_o2_e.lower_bound=0
    EcN_model.reactions.BIOMASS_EcN_iHM1533_core_59p80M.add_metabolites({'pg160_p': 0.0148265788551,
                                                                         'pg161_p': 0.013732749318999999,
                                                                         #'pe181_p': 0.0132509524519,
                                                                         'pg181_p': 0.00909808994019})
    for rxn in ['PGSA140', 'PGSA160', 'PGSA161', 'PGSA141', 'PGSA181', 'PGSA120', 'PGSA180']:
        EcN_model.reactions.get_by_id(rxn).lower_bound=0
        EcN_model.reactions.get_by_id(rxn).upper_bound=0
        sol = EcN_model.slim_optimize()
        print(rxn, sol)
        
with k12_model:
    M1(k12_model)
    k12_model.reactions.EX_o2_e.lower_bound=0
    for rxn in ['PGSA140', 'PGSA160', 'PGSA161', 'PGSA141', 'PGSA181', 'PGSA120', 'PGSA180']:
        k12_model.reactions.get_by_id(rxn).lower_bound=0
        k12_model.reactions.get_by_id(rxn).upper_bound=0
        sol = k12_model.slim_optimize()
        print(rxn, sol)

In [ ]:
EcN_model.metabolites.pg160_p

In [ ]:
EcN_model.reactions.BIOMASS_EcN_iHM1533_core_59p80M.metabolites

In [ ]:
EcN_model.metabolites.pg160_p

## K12 ess
### ZupT
heavy metal divalent cation transporter ZupT
https://ecocyc.org/gene?orgid=ECOLI&id=EG11167

### proB (corrected)
glutamate 5-kinase
https://ecocyc.org/gene?orgid=ECOLI&id=EG10768

### proA (corrected)
glutamate-5-semialdehyde dehydrogenase
https://ecocyc.org/gene?orgid=ECOLI&id=EG10767

## EcN ess
### argI/argF (corrected)
ornithine carbamoyltransferase
https://ecocyc.org/gene?orgid=ECOLI&id=EG10069

### leuB
3-isopropylmalate dehydrogenase
https://ecocyc.org/gene?orgid=ECOLI&id=EG11577

### can
carbonic anhydrase 2
https://ecocyc.org/gene?orgid=ECOLI&id=EG12319

In [ ]:
## zupT
# EcN_model.genes.CIW80_09665
# EcN_model.reactions.COBALT2tpp
# EcN_model.metabolites.cobalt2_c
# EcN_model.reactions.COBALT2t3pp

# EcN_model.reactions.CU2tpp
# EcN_model.metabolites.cu2_p 
# EcN_model.reactions.CU2abcpp

## proA/B
# EcN_model.genes.CIW80_19485
# EcN_model.genes.CIW80_19490
# EcN_model.reactions.G5SD
# EcN_model.metabolites.glu5sa_c 
# EcN_model.reactions.NACODA
# EcN_model.reactions.ACODA
# EcN_model.metabolites.orn_c

## argI/argF
# EcN_model.genes.CIW80_16625
# EcN_model.reactions.OCBT

## leuB
# EcN_model.genes.CIW80_18070
# EcN_model.reactions.IPMD

# can
# EcN_model.genes.CIW80_18385
# EcN_model.reactions.HCO3E

In [ ]:
# Check effect of specific reactions encoded by these genes
with k12_model:
    k12_model.remove_reactions(['OCBT'])
    growth = k12_model.slim_optimize()
    print(growth)

# Comparison double lethals

In [ ]:
# Create a list of all non-essential genes
an_non_k12 = del_result_k12[del_result_k12['K12_growth'] == 1].index.tolist()

# Run the double gene deletion with only the non-essential genes
with k12_model:
    M1(k12_model)
    
    # Make anaerobic
    medium = k12_model.medium
    medium['EX_o2_e']= 0.0
    k12_model.medium = medium
    
    double_an_k12 = double_gene_deletion(k12_model, an_non_k12, return_frame=True).round(4)
    
double_an_k12.head()

In [ ]:
double_an_k12.to_csv('../tables/gene_ess_double_ess_k12_raw.csv')

##### Load the tables not to rerun the double_gene_deletion

In [ ]:
# Load if needed
double_an_k12 = pd.read_csv('../tables/gene_ess_double_ess_k12_raw.csv')
double_an_k12.set_index('Unnamed: 0', inplace=True)

double_an_k12.head()

In [ ]:
# Get binary representation of gene essentiality
double_an_k12['binary']= double_an_k12['growth'].map(lambda x: 0 if x < (0.05 * k12_growth) else 1)

# Check whether there are genes for which the status is not "optimal"
inf_double_k12 = double_an_k12[double_an_k12.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in inf_double_k12:
    double_an_k12.loc[gene, 'binary'] = 0
    
double_an_k12[double_an_k12.status != 'optimal']

In [ ]:
double_ess_k12 = double_an_k12[double_an_k12.status == 'optimal']
double_ess_k12 = double_ess_k12[double_ess_k12['binary'] == 0]

print(len(double_ess_k12))
double_ess_k12.head()

In [ ]:
double_ess_k12[double_ess_k12['ids'].str.contains('CIW80_16625')]

In [ ]:
# Add an empty row for EcN as placeholder
double_ess_k12['EcN'] = 0

gene_list = ['gene_1', 'gene_2']
set_K12

for k12_loc in double_ess_k12.index.to_list():
    k12_id = double_ess_k12.loc[k12_loc, 'ids']
    genes = k12_id.split("', '")
    gene_list[0] = genes[0][2:]
    gene_list[1] = genes[1][:-2]
    
    # Convert to EcN gene IDs
    EcN_genes = []
    for gene in gene_list:
        try:
            EcN_gene = geneIDs_ess[geneIDs_ess['U00096'] == gene].index[0]
            EcN_genes.append(str(EcN_gene))
        except:
            print('Error', gene, 'of combination', gene_list)
            EcN_genes.append('Missing')
        
    double_ess_k12.loc[k12_loc,'EcN'] = str(EcN_genes)
    set_K12.add(frozenset(EcN_genes))
    
print(len(double_ess_k12))

In [ ]:
# Save the resulting dataframe as .csv
double_ess_k12.to_csv('../tables/gene_ess_double_ess_k12.csv')

#### Compare to EcN

In [ ]:
# Compare to EcN
double_ess_an = pd.read_csv('../tables/gene_ess_double_ess_an.csv')
double_ess_an.set_index('Unnamed: 0', inplace=True)

set_EcN = set()

for EcN_loc in double_ess_an.index.tolist():
    gene_list = double_ess_an.loc[EcN_loc, 'ids'].split("', '")
    gene_1 = gene_list[0][2:]
    gene_2 = gene_list[1][:-2]
    double_ess_an.loc[EcN_loc, 'genes_an'] = "['" + gene_1 + "', '" + gene_2 + "']"
    set_EcN.add(frozenset([gene_1, gene_2]))

diff_EcN = set_EcN - set_K12
diff_K12 = set_K12 - set_EcN
similar = set.intersection(set_EcN, set_K12)

print('Total number of double lethals specific to EcN:', len(diff_EcN), 'and to K12:', len(diff_K12))
print('The difference is:', len(diff_EcN), len(diff_K12), len(similar))

In [ ]:
diff_K12

In [ ]:
geneIDs_sub = orthoIDs_matrix.loc[:, 'U00096']
geneIDs_sub

In [ ]:
comparison_df = pd.DataFrame(columns=['K12_gene_1', 'K12_gene_2', 'EcN_gene_1', 'EcN_gene_2','growth', 'growth_1', 'growth_2'])

loc = 0

for gene_set in diff_EcN:
    gene_list = []
    gene_loc = 1
    for gene in gene_set:
        # Add EcN gene ID
        comparison_df.loc[loc, 'EcN_gene_%s'%gene_loc] = gene
        
        # Get also the k12 gene ID
        try:
            K12_gene = geneIDs_sub.loc[gene]        
        except:
            K12_gene = 'missing'
        
        comparison_df.loc[loc, 'K12_gene_%s'%gene_loc] = K12_gene
        
                # Check the knockout with K12
        with k12_model:
            M1(k12_model)

            # Make anaerobic
            medium = k12_model.medium
            medium['EX_o2_e']= 0.0
            k12_model.medium = medium
            try:
                k12_model.genes.get_by_id(K12_gene).knock_out()
                comparison_df.loc[loc, 'growth_%s'%gene_loc] = k12_model.slim_optimize()
            except:
                print(gene)
        
        # Go to the second gene
        gene_loc += 1
        gene_list.append(K12_gene)
#         gene_list.append(gene)
    
#     # Check the knockout with K12
    with k12_model:
        M1(k12_model)

        # Make anaerobic
        medium = k12_model.medium
        medium['EX_o2_e']= 0.0
        k12_model.medium = medium
        for gene in gene_list:
            try:
                k12_model.genes.get_by_id(gene).knock_out()
                comparison_df.loc[loc, 'growth' ] = k12_model.slim_optimize()
            except:
                print(gene_list)
    
#     # Check the knockout with EcN
#     with EcN_model:
#         M1(EcN_model)

#         # Make anaerobic
#         medium = EcN_model.medium
#         medium['EX_o2_e']= 0.0
#         EcN_model.medium = medium
#         print(gene_list)
#         for gene in gene_list:
#             EcN_model.genes.get_by_id(gene).knock_out()
#             comparison_df.loc[loc, 'growth'] = EcN_model.slim_optimize()        

    # Go to next row    
    loc+= 1
        
comparison_df

In [ ]:
comparison_df[:50]
# comparison_df[20:67]

In [ ]:
# Try by myself
gene_del_df = pd.DataFrame(columns=['mix', 'gene_1', 'gene_2', 'growth'])

gene_list = []

for gene in k12_model.genes:
    gene_list.append(gene.id)

# gene_del_df.loc[:,'gene_1'] = gene_list
# gene_del_df.set_index('gene_1', inplace=True)
loc=0

for gene in gene_list:
    for gene_2 in gene_list:
        if gene == gene_2:
            pass
        else:
            with k12_model:        
                # Media
                M1(k12_model)
                medium = k12_model.medium
                medium['EX_o2_e']= 0.0
                k12_model.medium = medium

                # Knockouts
                k12_model.genes.get_by_id(gene).knock_out()
                k12_model.genes.get_by_id(gene_2).knock_out()

                # Save in df
                gene_del_df.loc[loc, 'mix' ] = str(gene + ', ' + gene_2)
                gene_del_df.loc[loc, 'gene_1' ] = gene                
                gene_del_df.loc[loc, 'gene_2' ] = gene_2
                gene_del_df.loc[loc, 'growth' ] = k12_model.slim_optimize()
                
                # Next row
                loc += 1

    del gene_list[0]
    
gene_del_df

In [ ]:
gene_del_df.to_csv('../tables/double_gene_del_k12.csv')

### Investigate the difference in double lethals

In [ ]:
EcN_model.genes.CIW80_25730
# EcN_model.genes.CIW80_14630

In [ ]:
# Check the double lethal only for aerobic
for rxn_1 in k12_model.genes.b1378.reactions:
    for rxn_2 in k12_model.genes.b3924.reactions:
        with k12_model:
            M1(k12_model)
#             k12_model.reactions.EX_o2_e.lower_bound=0   
            k12_model.remove_reactions([rxn_1.id, rxn_2.id])
            print(rxn_1.id, rxn_2.id, k12_model.slim_optimize())

In [ ]:
# Check the double lethal only for aerobic
for rxn_1 in EcN_model.genes.CIW80_25730.reactions:
    for rxn_2 in EcN_model.genes.CIW80_14630.reactions:
        with EcN_model:
            M1(EcN_model)
#             EcN_model.reactions.EX_o2_e.lower_bound=0    
            EcN_model.remove_reactions([rxn_1.id, rxn_2.id])
            print(rxn_1.id, rxn_2.id, EcN_model.slim_optimize())

In [ ]:
# Check the reactions
k12_model.reactions.FLDR2
k12_model.reactions.POR5

# Check the metabolites and compare with EcN
EcN_model.metabolites.flxso_c.summary()
k12_model.metabolites.flxso_c.summary()

In [ ]:
# The only difference in consumption/production of these flavodoxin metabolites is MECDPDH5
with k12_model:
    M1(k12_model)
#     k12_model.reactions.EX_o2_e.lower_bound=0   
    k12_model.remove_reactions(['MECDPDH5'])
    print(k12_model.slim_optimize())

In [ ]:
# Check the difference in metabolism of k12 and EcN
k12_model.reactions.MECDPDH5
EcN_model.metabolites.get_by_id('2mecdp_c')
k12_model.metabolites.get_by_id('2mecdp_c')

In [ ]:
EcN_model.reactions.MECDPDH2

The EcN model includes another reaction, MECDPH2, which used NADH as electron acceptor, instead of flavodoxin